<a href="https://colab.research.google.com/github/kazeidk/GDrive_Turbo_Copy/blob/main/GDrive_Turbo_Copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 GDrive Turbo Copy v1.0

**⚡ Copy folder Google Drive → Google Drive | Nhanh • Ổn định • Hỗ trợ 2000GB+**

---

### 👨‍💻 Tác giả: Nguyễn Ngọc Anh Tú
| 🌐 | Link |
|---|---|
| 📢 **Kênh thông báo** | [Messenger Channel](https://www.messenger.com/channel/NguyenNgocAnhTu.VN) |
| 📘 Facebook | [NguyenNgocAnhTu.VN](https://www.facebook.com/NguyenNgocAnhTu.VN) |
| ✈️ Telegram | [@NguyenNgocAnhTu](https://t.me/NguyenNgocAnhTu) |
| 🐙 GitHub | [kazeidk](https://github.com/kazeidk) |

---

### ✨ Tính năng chính
| Icon | Tính năng | Mô tả |
|:---:|---|---|
| ⚡ | **Turbo Copy** | Copy trực tiếp, không scan trước |
| 📊 | **Real-time Stats** | Hiển thị file, dung lượng, tốc độ |
| 🎯 | **Exact Match** | Kiểm tra file chính xác 100% |
| 🔄 | **Auto Resume** | Tự tiếp tục khi timeout |
| 🔁 | **Smart Retry** | Retry 5 lần với exponential backoff |
| 💾 | **Checkpoint** | Lưu tiến độ + backup tự động |
| 📝 | **Full Log** | Ghi log chi tiết để xem lại |
| 🔍 | **Smart Filter** | Lọc theo đuôi file linh hoạt |
| 📄 | **Export Docs** | Xuất Google Docs sang PDF |
| 🔔 | **Sound Alert** | Âm thanh thông báo khi xong |
| 🧹 | **Auto GC** | Tự dọn RAM, tránh crash |

---

### 📖 Hướng dẫn nhanh
```
1️⃣ Chạy Cell 1 → Nhập link nguồn & đích
2️⃣ Chạy Cell 2 → Bắt đầu copy
3️⃣ Timeout? → Chạy lại Cell 2 (tự resume)
```

### 💡 Pro Tips
- 🌙 Chạy **ban đêm** = ít rate limit
- 💪 Dùng **Colab Pro** + **High RAM** cho folder lớn
- 📋 Xem log: `/content/gdrive_copy.log`

In [ ]:
#@title 1️⃣ 📝 Nhập thông tin
from ipywidgets import widgets, Layout
from IPython.display import display, HTML

style = {'description_width': '140px'}
layout = Layout(width='580px')

display(HTML('''
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 15px; border-radius: 10px; margin-bottom: 15px;">
    <h2 style="color: white; margin: 0;">🚀 GDrive Turbo Copy v1.0</h2>
    <p style="color: #e0e0e0; margin: 5px 0 0 0;">Copy folder Google Drive siêu nhanh</p>
</div>
'''))

dest_text = widgets.Text(description='📁 Folder đích:', placeholder='Link folder Google Drive của bạn', style=style, layout=layout)
source_text = widgets.Text(description='📂 Folder nguồn:', placeholder='Link folder cần copy (Shared Drive OK)', style=style, layout=layout)

display(HTML('<h4>📌 Thông tin bắt buộc:</h4>'))
display(dest_text, source_text)

exclude_str_text = widgets.Text(description='🚫 Bỏ qua chứa:', value='', placeholder='VD: .tmp, backup, test', style=style, layout=layout)
include_ext_text = widgets.Text(description='✅ Chỉ lấy đuôi:', value='', placeholder='VD: .mp4, .pdf, .zip', style=style, layout=layout)
exclude_ext_text = widgets.Text(description='❌ Bỏ qua đuôi:', value='', placeholder='VD: .tmp, .log, .bak', style=style, layout=layout)

display(HTML('<h4>🔍 Bộ lọc (tùy chọn):</h4>'))
display(exclude_str_text, include_ext_text, exclude_ext_text)

skip_exists_cb = widgets.Checkbox(description='⏭️ Bỏ qua file đã có (khuyến nghị)', value=True, indent=False, layout=Layout(width='320px'))
export_gdocs_cb = widgets.Checkbox(description='📄 Export Google Docs → PDF', value=False, indent=False, layout=Layout(width='320px'))
dry_run_cb = widgets.Checkbox(description='👁️ Dry-run (chỉ xem, không copy)', value=False, indent=False, layout=Layout(width='320px'))

display(HTML('<h4>⚙️ Cài đặt:</h4>'))
display(skip_exists_cb, export_gdocs_cb, dry_run_cb)

display(HTML('''
<div style="background: #d4edda; padding: 10px; border-radius: 8px; margin-top: 15px; border-left: 4px solid #28a745;">
    <b>✅ Nhập xong?</b> → Chạy <b>Cell 2</b> để bắt đầu copy!
</div>
'''))

In [ ]:
#@title 2️⃣ 🚀 Run - GDrive Turbo Copy
import os, time, re, json, logging, warnings, gc, shutil, sys, io
from datetime import datetime
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload, MediaIoBaseUpload
from google.colab import auth
from IPython.display import Audio, display, HTML
import numpy as np

warnings.filterwarnings('ignore')
logging.getLogger('googleapiclient.discovery_cache').setLevel(logging.ERROR)
logging.getLogger('google_auth_httplib2').setLevel(logging.ERROR)

class GDriveTurboCopy:
    VERSION = '1.0'
    EXPORT_MIMES = {
        'application/vnd.google-apps.document': ('application/pdf', '.pdf'),
        'application/vnd.google-apps.spreadsheet': ('application/pdf', '.pdf'),
        'application/vnd.google-apps.presentation': ('application/pdf', '.pdf'),
        'application/vnd.google-apps.drawing': ('application/pdf', '.pdf'),
    }
    
    def __init__(self):
        self.total_mb = self.copied = self.skipped = self.errors = self.exported = 0
        self.error_files, self.shortcuts, self.done_keys = [], [], set()
        self.exclude_str, self.include_ext, self.exclude_ext = [], [], []
        self.dry_run = False
        self.skip_exists = True
        self.export_gdocs = False
        self.checkpoint = '/content/checkpoint.json'
        self.checkpoint_backup = '/content/checkpoint.json.bak'
        self.log_file = '/content/gdrive_copy.log'
        self.svc = None
        self.start_time = self.last_gc = self.last_progress = 0
        self.source_url = self.dest_url = self.current_path = ''
        self._init_log()

    def _init_log(self):
        self.logger = logging.getLogger('GDriveCopy')
        self.logger.setLevel(logging.INFO)
        self.logger.handlers = []
        fh = logging.FileHandler(self.log_file, encoding='utf-8')
        fh.setFormatter(logging.Formatter('[%(asctime)s] %(message)s', '%H:%M:%S'))
        self.logger.addHandler(fh)

    def log(self, msg, also_print=True):
        self.logger.info(msg)
        if also_print: print(f'[{datetime.now().strftime("%H:%M:%S")}] {msg}')

    def load_cp(self):
        for cp_file in [self.checkpoint, self.checkpoint_backup]:
            try:
                if os.path.exists(cp_file):
                    with open(cp_file, 'r') as f: d = json.load(f)
                    self.done_keys = set(d.get('k', []))
                    self.total_mb, self.copied, self.skipped = d.get('m', 0), d.get('c', 0), d.get('s', 0)
                    self.exported, self.error_files, self.shortcuts = d.get('exp', 0), d.get('e', []), d.get('shortcuts', [])
                    if self.done_keys:
                        self.log(f'🔄 Resume: {len(self.done_keys):,} files ({self.total_mb/1024:.1f}GB)')
                    return True
            except: pass
        return False

    def save_cp(self):
        try:
            data = {'k': list(self.done_keys), 'm': self.total_mb, 'c': self.copied, 's': self.skipped,
                    'exp': self.exported, 'e': self.error_files[-100:], 'shortcuts': self.shortcuts[-50:],
                    'src': self.source_url, 'dst': self.dest_url, 't': datetime.now().isoformat(), 'v': self.VERSION}
            if os.path.exists(self.checkpoint): shutil.copy2(self.checkpoint, self.checkpoint_backup)
            with open(self.checkpoint, 'w') as f: json.dump(data, f)
        except: pass

    def auth(self):
        self.log('🔐 Đang xác thực...')
        auth.authenticate_user()
        self.svc = build('drive', 'v3')
        self.log('✅ Xác thực OK!')

    def play_sound(self, success=True):
        try:
            sr = 22050
            freqs, durs = ([523, 659, 784], [0.15, 0.15, 0.2]) if success else ([400, 300], [0.2, 0.3])
            sounds = [np.sin(2 * np.pi * f * np.linspace(0, d, int(sr * d))) * np.exp(-3 * np.linspace(0, d, int(sr * d))) for f, d in zip(freqs, durs)]
            display(Audio(np.concatenate(sounds), rate=sr, autoplay=True))
        except: pass

    def fmt_time(self, s):
        if s < 60: return f'{int(s)}s'
        if s < 3600: return f'{int(s//60)}m{int(s%60)}s'
        return f'{int(s//3600)}h{int((s%3600)//60)}m'

    def fmt_size(self, mb):
        if mb < 1024: return f'{mb:.1f}MB'
        if mb < 1024*1024: return f'{mb/1024:.2f}GB'
        return f'{mb/(1024*1024):.2f}TB'

    def maybe_gc(self):
        if time.time() - self.last_gc > 60: gc.collect(); self.last_gc = time.time()

    def show_progress(self):
        now = time.time()
        if now - self.last_progress < 2: return
        self.last_progress = now
        elapsed = now - self.start_time
        if elapsed < 1: return
        spd = self.total_mb / elapsed
        total = self.copied + self.skipped + self.exported
        sys.stdout.write(f'\r⚡ {total:,} files │ 💾 {self.fmt_size(self.total_mb)} │ 🚀 {spd:.1f}MB/s │ ⏱️ {self.fmt_time(elapsed)}' + ' '*10)
        sys.stdout.flush()

    def ok_ext(self, name):
        if not name: return True
        ext = os.path.splitext(name)[1].lower()
        if self.include_ext: return ext in self.include_ext
        if self.exclude_ext: return ext not in self.exclude_ext
        return True

    def clean_name(self, name):
        if not name: return 'unnamed'
        for c in '\\/:*?\"<>|': name = name.replace(c, '_')
        return ''.join(c for c in name if ord(c) >= 32)[:200].strip() or 'unnamed'

    def escape_q(self, s): return s.replace('\\', '\\\\').replace("'", "\\'")

    def call(self, fn, retry=5):
        for i in range(retry):
            try: return fn()
            except HttpError as e:
                if e.resp.status == 404: return None
                if e.resp.status in [403, 429, 500, 502, 503, 504]:
                    wait = min(3 * (2 ** i), 60)
                    self.log(f'⚠️ API {e.resp.status}, retry {i+1}/{retry} sau {wait}s...')
                    time.sleep(wait)
                else: raise
            except Exception as e:
                if i < retry - 1: time.sleep(min(2 * (2 ** i), 30))
                else: raise
        return None

    def ls(self, fid):
        items, token = [], None
        q = f"'{fid}' in parents and trashed=false"
        for s in self.exclude_str: q += f" and not name contains '{self.escape_q(s)}'"
        while True:
            r = self.call(lambda: self.svc.files().list(q=q, pageToken=token, pageSize=1000,
                fields='files(id,name,mimeType,size),nextPageToken', supportsAllDrives=True, includeItemsFromAllDrives=True).execute())
            if not r: break
            for f in r.get('files', []):
                mime, name = f.get('mimeType', ''), f.get('name', '')
                if mime == 'application/vnd.google-apps.shortcut':
                    self.shortcuts.append({'name': name, 'id': f['id'], 'path': self.current_path}); continue
                is_folder = mime == 'application/vnd.google-apps.folder'
                is_exportable = mime in self.EXPORT_MIMES
                is_gfile = mime.startswith('application/vnd.google-apps.')
                if is_folder or (is_exportable and self.export_gdocs) or (not is_gfile and self.ok_ext(name)): items.append(f)
            token = r.get('nextPageToken')
            if not token: break
        return items

    def exists(self, did, name):
        if not self.skip_exists: return None
        try:
            r = self.call(lambda: self.svc.files().list(q=f"'{did}' in parents and name='{self.escape_q(name)}' and trashed=false",
                fields='files(id)', supportsAllDrives=True, pageSize=1).execute())
            return r['files'][0]['id'] if r and r.get('files') else None
        except: return None

    def export_doc(self, did, src):
        fid, name, mime = src['id'], src.get('name', 'unnamed'), src.get('mimeType', '')
        if mime not in self.EXPORT_MIMES: return
        exp_mime, ext = self.EXPORT_MIMES[mime]
        fname = self.clean_name(name) + ext
        key = f"{did}:{fid}:exp"
        if key in self.done_keys: self.skipped += 1; return
        if self.dry_run: self.log(f'👁️ [DRY] Export: {fname}'); self.exported += 1; return
        if self.skip_exists and self.exists(did, fname): self.skipped += 1; self.done_keys.add(key); return
        try:
            buf = io.BytesIO()
            dl = MediaIoBaseDownload(buf, self.svc.files().export_media(fileId=fid, mimeType=exp_mime))
            done = False
            while not done: _, done = dl.next_chunk()
            buf.seek(0)
            r = self.call(lambda: self.svc.files().create(body={'name': fname, 'parents': [did]},
                media_body=MediaIoBaseUpload(buf, mimetype=exp_mime, resumable=True), fields='id,size', supportsAllDrives=True).execute())
            if r:
                mb = int(r.get('size', 0)) / (1024*1024)
                self.total_mb += mb; self.exported += 1; self.done_keys.add(key)
                self.log(f'📄 [{self.exported}] {fname[:35]} ({self.fmt_size(mb)})')
                self.show_progress()
        except Exception as e:
            self.errors += 1; self.error_files.append({'name': name, 'id': fid, 'error': f'Export: {str(e)[:80]}', 'path': self.current_path})

    def cp(self, did, src):
        fid, name, mime = src['id'], src.get('name', 'unnamed'), src.get('mimeType', '')
        if mime in self.EXPORT_MIMES:
            if self.export_gdocs: self.export_doc(did, src)
            return
        fname = self.clean_name(name)
        mb = int(src.get('size', 0)) / (1024*1024)
        key = f"{did}:{fid}"
        if key in self.done_keys: self.skipped += 1; self.show_progress(); return
        if self.dry_run: self.log(f'👁️ [DRY] {fname} ({self.fmt_size(mb)})'); self.total_mb += mb; self.copied += 1; self.show_progress(); return
        if self.skip_exists and self.exists(did, fname): self.skipped += 1; self.done_keys.add(key); self.show_progress(); return
        try:
            t0 = time.time()
            r = self.call(lambda: self.svc.files().copy(body={'parents': [did], 'name': fname}, fileId=fid, supportsAllDrives=True).execute(), retry=5)
            if r:
                spd = mb / max(time.time() - t0, 0.1)
                self.total_mb += mb; self.copied += 1; self.done_keys.add(key)
                self.log(f'✅ [{self.copied:,}] {fname[:35]} ({self.fmt_size(mb)}, {spd:.0f}MB/s)', also_print=False)
                self.show_progress(); self.maybe_gc()
        except Exception as e:
            self.errors += 1; self.error_files.append({'name': name, 'id': fid, 'error': str(e)[:100], 'path': self.current_path})
            self.log(f'❌ {fname[:35]}: {str(e)[:40]}')

    def mkdir(self, did, name):
        name = self.clean_name(name)
        eid = self.exists(did, name)
        if eid: return eid
        if self.dry_run: return f'dry_{name}'
        try:
            r = self.call(lambda: self.svc.files().create(body={'name': name, 'mimeType': 'application/vnd.google-apps.folder', 'parents': [did]},
                fields='id', supportsAllDrives=True).execute())
            if r: self.log(f'📁 Tạo: {name}'); return r['id']
        except Exception as e: self.log(f'❌ Mkdir {name}: {e}')
        return None

    def copy_dir(self, did, sid, path='', depth=0):
        self.current_path = path
        items = self.ls(sid)
        if not items: return
        folders = [f for f in items if f.get('mimeType') == 'application/vnd.google-apps.folder']
        files = [f for f in items if f.get('mimeType') != 'application/vnd.google-apps.folder']
        print(f'\n{"  "*depth}📊 {path or "Root"}: {len(files)} files, {len(folders)} folders')
        for i, f in enumerate(files):
            self.cp(did, f)
            if (i + 1) % 20 == 0: self.save_cp()
        if files: self.save_cp()
        for folder in folders:
            fname = folder.get('name', 'unnamed')
            new_path = f'{path}/{fname}' if path else fname
            print(f'\n{"  "*depth}📂 → {fname}')
            sub = self.mkdir(did, fname)
            if sub and not sub.startswith('dry_'): self.copy_dir(sub, folder['id'], new_path, depth + 1)

    def get_id(self, url):
        if not url: return None
        m = re.search(r'[-\w]{25,}', url)
        return m.group(0) if m else None

    def summary(self, elapsed):
        print('\n')
        self.log('═' * 50)
        self.log('📊 KẾT QUẢ:')
        self.log(f'   ✅ Copy: {self.copied:,} files')
        if self.exported: self.log(f'   📄 Export: {self.exported:,} docs')
        self.log(f'   ⏭️ Skip: {self.skipped:,} files')
        self.log(f'   ❌ Lỗi: {self.errors} files')
        if self.shortcuts: self.log(f'   🔗 Shortcuts: {len(self.shortcuts)}')
        self.log(f'   💾 Tổng: {self.fmt_size(self.total_mb)}')
        self.log(f'   ⏱️ Time: {self.fmt_time(elapsed)}')
        if elapsed > 0: self.log(f'   🚀 Speed: {self.total_mb/elapsed:.1f}MB/s')
        self.log('═' * 50)
        if self.error_files:
            self.log(f'\n📋 {len(self.error_files)} file lỗi:')
            for ef in self.error_files[:5]: self.log(f"   • {ef['name'][:40]}")
            if len(self.error_files) > 5: self.log(f'   ... +{len(self.error_files)-5} khác')

    def run(self, dest, src):
        self.source_url, self.dest_url = src, dest
        self.start_time = self.last_gc = time.time()
        display(HTML('<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 12px; border-radius: 8px; color: white;"><h3 style="margin:0;">🚀 GDRIVE TURBO COPY v1.0</h3></div>'))
        print()
        self.auth()
        self.load_cp()
        did, sid = self.get_id(dest), self.get_id(src)
        if not did or not sid: self.log('❌ URL không hợp lệ!'); return
        try:
            si = self.svc.files().get(fileId=sid, supportsAllDrives=True, fields='name').execute()
            di = self.svc.files().get(fileId=did, supportsAllDrives=True, fields='name').execute()
        except Exception as e: self.log(f'❌ Không truy cập được: {e}'); return
        self.log(f'📂 Nguồn: {si.get("name", "?")}')
        self.log(f'📁 Đích: {di.get("name", "?")}')
        if self.export_gdocs: self.log('📄 Export Docs: ON')
        if self.dry_run: self.log('👁️ DRY-RUN MODE')
        self.log('─' * 50)
        new_dest = self.mkdir(did, si.get('name', 'Copy'))
        if not new_dest: self.log('❌ Không tạo được folder!'); self.play_sound(False); return
        self.copy_dir(new_dest, sid, si.get('name', 'Copy'))
        self.save_cp()
        elapsed = time.time() - self.start_time
        self.summary(elapsed)
        if self.errors == 0: self.log('🎉 HOÀN TẤT!'); self.play_sound(True)
        else: self.log(f'⚠️ Có {self.errors} lỗi. Chạy lại để retry.'); self.play_sound(False)
        self.log(f'\n📝 Log: {self.log_file}')

# === RUN ===
print('🔧 Khởi tạo...\n')
dl = GDriveTurboCopy()
dl.exclude_str = [s.strip() for s in exclude_str_text.value.split(',') if s.strip()]
dl.include_ext = [('.' + s.strip().lstrip('.').lower()) for s in include_ext_text.value.split(',') if s.strip()]
dl.exclude_ext = [('.' + s.strip().lstrip('.').lower()) for s in exclude_ext_text.value.split(',') if s.strip()]
dl.skip_exists, dl.export_gdocs, dl.dry_run = skip_exists_cb.value, export_gdocs_cb.value, dry_run_cb.value

if not dest_text.value.strip(): print('❌ Chưa nhập folder đích!')
elif not source_text.value.strip(): print('❌ Chưa nhập folder nguồn!')
else: dl.run(dest_text.value.strip(), source_text.value.strip())

In [ ]:
#@title 3️⃣ 🗑️ Xóa checkpoint (copy lại từ đầu)
import os
from IPython.display import display, HTML

print('🗑️ Đang xóa checkpoint...\n')
deleted = False
for f in ['/content/checkpoint.json', '/content/checkpoint.json.bak']:
    if os.path.exists(f): os.remove(f); print(f'   ✅ Đã xóa: {f}'); deleted = True

if not deleted: print('   ℹ️ Không có checkpoint.')

display(HTML('<div style="background: #d4edda; padding: 10px; border-radius: 8px; margin-top: 10px; border-left: 4px solid #28a745;"><b>🔄 Sẵn sàng!</b> Chạy lại Cell 2 để copy từ đầu.</div>'))

In [ ]:
#@title 4️⃣ 📋 Xem Log & Lỗi
import os, json
from IPython.display import display, HTML

LOG_LINES = 100  # Số dòng log hiển thị

log_file = '/content/gdrive_copy.log'
cp_file = '/content/checkpoint.json'

# === LOG ===
if os.path.exists(log_file):
    with open(log_file, 'r', encoding='utf-8') as f: lines = f.readlines()
    display(HTML(f'<h3>📝 Log ({len(lines)} dòng, hiện {min(LOG_LINES, len(lines))} cuối)</h3>'))
    print('─' * 60)
    for line in lines[-LOG_LINES:]: print(line.rstrip())
    print('─' * 60)
else:
    print('ℹ️ Chưa có log. Chạy Cell 2 trước.')

# === ERRORS & SHORTCUTS ===
if os.path.exists(cp_file):
    with open(cp_file, 'r') as f: data = json.load(f)
    errors, shortcuts = data.get('e', []), data.get('shortcuts', [])
    
    if errors:
        display(HTML(f'<h3>❌ {len(errors)} File Lỗi</h3>'))
        for i, ef in enumerate(errors[:10], 1):
            print(f"{i}. {ef.get('name', '?')[:50]}")
            print(f"   📍 {ef.get('path', 'N/A')}")
            print(f"   ⚠️ {ef.get('error', 'N/A')[:60]}\n")
        if len(errors) > 10: print(f'... +{len(errors)-10} file khác')
        display(HTML('<div style="background: #fff3cd; padding: 8px; border-radius: 5px; border-left: 4px solid #ffc107;">💡 Chạy lại Cell 2 để retry</div>'))
    else:
        display(HTML('<div style="background: #d4edda; padding: 8px; border-radius: 5px; border-left: 4px solid #28a745;">✅ Không có file lỗi!</div>'))
    
    if shortcuts:
        display(HTML(f'<h3>🔗 {len(shortcuts)} Shortcuts (bỏ qua)</h3>'))
        for i, sc in enumerate(shortcuts[:10], 1): print(f"   {i}. {sc.get('name', '?')[:50]}")
        if len(shortcuts) > 10: print(f'   ... +{len(shortcuts)-10} khác')